In [ ]:
The "list index out of range" error typically occurs when trying to access an element in a list that doesn't exist. In this context, it's likely happening because pairs[0] might not exist if no cointegrated pairs were found.

We need to add error handling to ensure that pairs are available before proceeding. Here is the revised code with error handling and additional refinements:

### Step-by-Step Revised Implementation

### Step 1: Collect Historical Price Data

python
import yfinance as yf
import pandas as pd

# List of stock tickers
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "FB", "TSLA"]

# Download historical price data
data = yf.download(tickers, start="2015-01-01", end="2023-01-01")['Adj Close']


### Step 2: Identify Pairs with High Degree of Cointegration

python
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
import numpy as np

# Function to find cointegrated pairs
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    pairs = []
    for i in range(n):
        for j in range(i + 1, n):
            S1 = data.iloc[:, i]
            S2 = data.iloc[:, j]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:  # 5% significance level
                pairs.append((data.columns[i], data.columns[j]))
    return score_matrix, pvalue_matrix, pairs

scores, pvalues, pairs = find_cointegrated_pairs(data)
if not pairs:
    raise ValueError("No cointegrated pairs found")
print("Cointegrated pairs:", pairs)


### Step 3: Develop a Trading Algorithm

python
# Function to calculate z-score
def zscore(series):
    return (series - series.mean()) / np.std(series)

# Example pair
pair = pairs[0]
S1 = data[pair[0]]
S2 = data[pair[1]]

# Calculate the spread
spread = S1 - S2

# Calculate the z-score of the spread
zscore_spread = zscore(spread)

# Define thresholds for trading signals
entry_threshold = 2.0
exit_threshold = 0.5

# Generate signals
data['position'] = np.where(zscore_spread > entry_threshold, -1, 0)  # Short the spread
data['position'] = np.where(zscore_spread < -entry_threshold, 1, data['position'])  # Long the spread
data['position'] = np.where(np.abs(zscore_spread) < exit_threshold, 0, data['position'])  # Exit position

# Shift the position by 1 day to align with next day's returns
data['position'] = data['position'].shift(1)


### Step 4: Backtest the Algorithm

python
# Calculate daily returns
data['returns'] = data[pair[0]].pct_change() - data[pair[1]].pct_change()

# Calculate strategy returns
data['strategy_returns'] = data['position'] * data['returns']

# Calculate cumulative returns
data['cumulative_returns'] = (1 + data['strategy_returns']).cumprod()

# Plot cumulative returns
data['cumulative_returns'].plot(figsize=(12, 6))


### Step 5: Optimize the Algorithm

python
from itertools import product

def optimize_strategy(data, pair):
    best_sharpe = -np.inf
    best_params = None
    thresholds = np.arange(0.5, 3.5, 0.5)
    for entry_threshold, exit_threshold in product(thresholds, repeat=2):
        # Calculate z-score of the spread
        spread = data[pair[0]] - data[pair[1]]
        zscore_spread = zscore(spread)

        # Generate signals
        data['position'] = np.where(zscore_spread > entry_threshold, -1, 0)
        data['position'] = np.where(zscore_spread < -entry_threshold, 1, data['position'])
        data['position'] = np.where(np.abs(zscore_spread) < exit_threshold, 0, data['position'])
        data['position'] = data['position'].shift(1)

        # Calculate strategy returns
        data['strategy_returns'] = data['position'] * (data[pair[0]].pct_change() - data[pair[1]].pct_change())

        # Calculate Sharpe ratio
        sharpe_ratio = data['strategy_returns'].mean() / data['strategy_returns'].std() * np.sqrt(252)

        if sharpe_ratio > best_sharpe:
            best_sharpe = sharpe_ratio
            best_params = (entry_threshold, exit_threshold)

    return best_params, best_sharpe

best_params, best_sharpe = optimize_strategy(data, pair)
print("Best parameters:", best_params)
print("Best Sharpe ratio:", best_sharpe)


### Explanation

1. *Collecting Data*: Use yfinance to download historical price data for a list of stock tickers.
2. *Identifying Cointegrated Pairs*: Use the cointegration test to find pairs of stocks that have a long-term equilibrium relationship. Added error handling for cases when no pairs are found.
3. *Trading Algorithm*: Create a simple trading strategy based on the z-score of the price spread between the two assets. The algorithm generates buy/sell signals when the spread diverges significantly from the mean and closes the position when the spread reverts to the mean.
4. *Backtesting*: Backtest the strategy by calculating daily returns, applying the trading signals, and plotting cumulative returns.
5. *Optimization*: Test different thresholds for entering and exiting trades to find the parameters that maximize the Sharpe ratio, a measure of risk-adjusted return.

By following these steps and adding error handling, you can develop a pairs trading strategy, backtest it, and optimize it for better performance.